In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
from librep.base.estimator import Estimator
%load_ext autoreload
%autoreload 2
from standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../../../../../../experiment_executor/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=True)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-01 16:33:44.392335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 16:33:44.430135: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 16:33:44.430176: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 16:33:44.430201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea

In [2]:
X_train, y_train,X_test, y_test,X_val, y_val=get_data("UCI",['accel','gyro'],False)
input_shape=X_train[0].shape
print(input_shape)

(60, 6)


In [3]:
#import simclr_utils as sm_ut
#transformations=sm_ut.transform_combinations(3,4)

#print(transformations)

In [4]:
from librep.transforms.simclr_full import SimCLR_full

batch_size_head = 256
decay_steps_head = 1000
epochs_head = 300
temperature_head = 0.77
input_shape=X_train[0].shape
n_components=60
print(input_shape)
transform_funcs=[1, 2, 3]
verbose=0
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#for transform_funcs in transformations:

#transformations=sm_ut.transform_combinations(3,4)
simclr_linear = SimCLR_full(dataset='motionsense',input_shape=input_shape,
                        n_components=n_components,
                        batch_size_head=batch_size_head,
                        transform_funcs=transform_funcs,
                        temperature_head=temperature_head, 
                        epochs_head=epochs_head,
                        save_model=False,
                        verbose=1,
                        total_epochs=50,
                        batch_size=32,
                        lr=0.001,
                           patience=10,
                           min_delta=0.001,
                            device='cuda',
                            save_reducer=False
                           )
#print(transform_funcs)
    
trained_simclr_model = simclr_linear.fit(X_train,y_train)


(60, 6)
cuda
exist model
Epoch [1/50] - Loss: 1.107007397002861
Val Accuracy: 0.3864
Epoch [2/50] - Loss: 0.818042492280241
Val Accuracy: 0.5640
Epoch [3/50] - Loss: 0.5805482111993383
Val Accuracy: 0.7459
Epoch [4/50] - Loss: 0.49832264761455725
Val Accuracy: 0.7459
Epoch [5/50] - Loss: 0.39431308893883815
Val Accuracy: 0.7769
Epoch [6/50] - Loss: 0.356502931870398
Val Accuracy: 0.7955
Epoch [7/50] - Loss: 0.32880944209020646
Val Accuracy: 0.7996
Epoch [8/50] - Loss: 0.3254986989205001
Val Accuracy: 0.7913
Epoch [9/50] - Loss: 0.32564888747989157
Val Accuracy: 0.7810
Epoch [10/50] - Loss: 0.32863355562335156
Val Accuracy: 0.7789
Epoch [11/50] - Loss: 0.30426704053018916
Val Accuracy: 0.7686
Epoch [12/50] - Loss: 0.3364693791162772
Val Accuracy: 0.8244
Epoch [13/50] - Loss: 0.2982412999282118
Val Accuracy: 0.7727
Epoch [14/50] - Loss: 0.29917283331761596
Val Accuracy: 0.8182
Epoch [15/50] - Loss: 0.3017507899002951
Val Accuracy: 0.8099
Epoch [16/50] - Loss: 0.2839813615943565
Val Accur

In [8]:
print(simclr_linear.model.base_model)

BaseModel(
  (conv1): Conv1d(6, 32, kernel_size=(24,), stride=(1,))
  (dropout1): Dropout(p=0.1, inplace=False)
  (conv2): Conv1d(32, 64, kernel_size=(16,), stride=(1,))
  (dropout2): Dropout(p=0.1, inplace=False)
  (conv3): Conv1d(64, 60, kernel_size=(8,), stride=(1,))
  (dropout3): Dropout(p=0.1, inplace=False)
  (global_max_pooling): AdaptiveMaxPool1d(output_size=1)
  (relu): ReLU()
)


In [9]:
data=simclr_linear.transform(X_train)
data_t=simclr_linear.transform(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, random_state=42)  
# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)

(2420, 60)
(2420, 60, 6)
0.9               precision    recall  f1-score   support

           0       0.79      0.79      0.79       138
           1       0.79      0.78      0.79       138
           2       0.95      0.98      0.96       138
           3       0.98      0.95      0.97       138
           4       0.99      1.00      0.99       138

    accuracy                           0.90       690
   macro avg       0.90      0.90      0.90       690
weighted avg       0.90      0.90      0.90       690



In [11]:
data=simclr_linear.transform(X_train)
data_t=simclr_linear.transform(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)

(2420, 60)
(2420, 60, 6)
0.8971014492753623               precision    recall  f1-score   support

           0       0.79      0.78      0.78       138
           1       0.77      0.79      0.78       138
           2       0.96      0.97      0.96       138
           3       0.98      0.95      0.96       138
           4       0.99      1.00      0.99       138

    accuracy                           0.90       690
   macro avg       0.90      0.90      0.90       690
weighted avg       0.90      0.90      0.90       690

